In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# import data and one_hot encode

In [49]:
# Training Parameters
learning_rate = 0.001 # optimize this
training_steps = 10000
batch_size = 10 # 128
display_step = 200

In [4]:
# Network Parameters
num_input = 5
    # 28 # MNIST data input (img shape: 28*28)
timesteps = 23
    # 28 # timesteps
num_hidden = 1 # ?
    # 128 # hidden layer num of features
num_classes = 2 # figure out how to do regression
    # 10 # MNIST total classes (0-9 digits)

In [5]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [6]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [50]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [8]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    # switch to adam optimizer
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [9]:
# read in full data
data= pd.read_csv('../GenomeCRISPR_full05112017.csv')

# read in sample data
#data= pd.read_csv('../GenomeCRISPR_full05112017_small.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
#should filter only to our pubmed 29083409
toppub_all_data = data[data['pubmed'] ==29083409]

In [11]:
# groupby celline and sequence
toppub_grouped = toppub_all_data.groupby(['sequence','symbol']).median()['effect'].reset_index()

In [12]:
# groupby sequence to get unique seqs
groupby_seq = toppub_grouped.groupby(['sequence']).count().reset_index()
groupby_gene = toppub_grouped.groupby(['symbol']).count().reset_index()

In [13]:
# keeping only sequence column
unique_seq_toppub = groupby_seq['sequence']
groupby_gene = groupby_gene['symbol']

In [14]:
# one_hot function

e_dict = {'A':[1,0,0,0], 'C':[0,1,0,0], 'G':[0,0,1,0], 'T':[0,0,0,1]}

def one_hot(x):
    return e_dict[x]

In [15]:
unique_seq_toppub.head()

0    AAAAAAATCCAGCAATGCAGAGG
1    AAAAAACCCGTAGATAGCCTCGG
2    AAAAAAGAAGAAAAAACCAGAGG
3    AAAAAAGCTCAAGAAGGAGGCGG
4    AAAAAAGGCTGTAAAAGCGTGGG
Name: sequence, dtype: object

In [16]:
# onehot encode the unique seqs
one_hot_un_seq = unique_seq_toppub.apply(lambda x: map(one_hot, x))

In [17]:
x = one_hot_un_seq

In [18]:
# reset indices
unique_seq_toppub = unique_seq_toppub.reset_index()
one_hot_un_seq = one_hot_un_seq.reset_index()
groupby_gene = groupby_gene.reset_index()

In [19]:
# append onehot to seqs
unique_seq_toppub['one_hot'] = one_hot_un_seq['sequence']

In [20]:
unique_seq_toppub.head()

,index,sequence,one_hot
0,0,AAAAAAATCCAGCAATGCAGAGG,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
1,1,AAAAAACCCGTAGATAGCCTCGG,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
2,2,AAAAAAGAAGAAAAAACCAGAGG,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
3,3,AAAAAAGCTCAAGAAGGAGGCGG,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."
4,4,AAAAAAGGCTGTAAAAGCGTGGG,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,..."


In [21]:
# merge one hot encoding to data
toppub_one_hot = toppub_grouped.merge(unique_seq_toppub,left_on="sequence",right_on="sequence", how="left")

In [22]:
# merge gene encoding to data
toppub_one_hot = toppub_one_hot.merge(groupby_gene,left_on="symbol",right_on="symbol", how="left")

In [23]:
toppub_one_hot.head()

,sequence,symbol,effect,index_x,one_hot,index_y
0,AAAAAAATCCAGCAATGCAGAGG,SHOC2,-1.0,0,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...",14001
1,AAAAAACCCGTAGATAGCCTCGG,NDUFA12,-1.0,1,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...",9936
2,AAAAAAGAAGAAAAAACCAGAGG,SDAD1,-5.0,2,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...",13701
3,AAAAAAGCTCAAGAAGGAGGCGG,FAM98A,1.0,3,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...",5316
4,AAAAAAGGCTGTAAAAGCGTGGG,ZNF253,-3.0,4,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...",17743


In [24]:
# create empty numpy matrix for storing one hot features
features = np.ndarray(shape=(toppub_one_hot.shape[0],23,5), dtype=int)

In [25]:
# append gene to each one hot nucleotide
for i in range(toppub_one_hot.shape[0]):
    for j in range(len(toppub_one_hot['one_hot'][i])):
        features[i][j] = toppub_one_hot['one_hot'][i][j] + [toppub_one_hot['index_y'][i]]

In [42]:
def convert_labels(y):
    if y > 3:
        return 1
    else:
        return 0

In [43]:
# construct labels matrix
labels = toppub_one_hot['effect']
labels = np.array(list(map(lambda x: convert_labels(x), labels)))
labels = np.eye(num_classes)[labels]

array([54383., 18480.])

In [46]:
# split into training and testing
msk = np.random.rand(len(features)) < 0.8

train_data = features[msk]
train_labels = labels[msk]

test_data = features[~msk]
test_labels = labels[~msk]

In [47]:
def get_train_data(batch_size, x, y):
    i = np.random.randint(0,x.shape[0], size=(batch_size))
    return np.array(x[i]), np.array(y[i])

In [51]:
x, y = train_data, train_labels
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = get_train_data(batch_size, x,y)
        # Reshape data to get 28 seq of 28 elements
#         batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        batch_y = batch_y.reshape((batch_size, num_classes))
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
#     test_len = 128
#     test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
#     test_label = mnist.test.labels[:test_len]
#     print("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 1.1450, Training Accuracy= 0.500
Step 200, Minibatch Loss= 0.5053, Training Accuracy= 0.800
Step 400, Minibatch Loss= 0.5006, Training Accuracy= 0.800
Step 600, Minibatch Loss= 0.6215, Training Accuracy= 0.700
Step 800, Minibatch Loss= 0.5058, Training Accuracy= 0.800
Step 1000, Minibatch Loss= 0.7243, Training Accuracy= 0.600
Step 1200, Minibatch Loss= 0.6187, Training Accuracy= 0.700
Step 1400, Minibatch Loss= 0.7266, Training Accuracy= 0.600
Step 1600, Minibatch Loss= 0.6145, Training Accuracy= 0.700
Step 1800, Minibatch Loss= 0.6155, Training Accuracy= 0.700
Step 2000, Minibatch Loss= 0.5077, Training Accuracy= 0.800
Step 2200, Minibatch Loss= 0.3973, Training Accuracy= 0.900
Step 2400, Minibatch Loss= 0.2878, Training Accuracy= 1.000
Step 2600, Minibatch Loss= 0.5045, Training Accuracy= 0.800
Step 2800, Minibatch Loss= 0.7282, Training Accuracy= 0.600
Step 3000, Minibatch Loss= 0.5093, Training Accuracy= 0.800
Step 3200, Minibatch Loss= 0.6146, Training Acc

In [52]:
sum(labels)

array([54383., 18480.])